In [1]:
import pickle
import torch
from collections import Counter

In [2]:
def get_conf_scores(emb_dir):   
    res = pickle.load( open(emb_dir, 'rb') )
    act, pred, tar = res['act'], res['pred'], res['tar']
    sm = torch.nn.functional.softmax(act, dim=1)
    acc = ( pred == tar ).sum() / len(pred)

    print('averaged conf:', sm.max(1)[0].mean().item())
    print('acc:', acc.item())
    pred_counts = Counter(pred.tolist())
    print('label dist:', [pred_counts[i] / len(tar) for i in range(act.shape[1])] )
    print()

    return sm.max(1)[0].tolist()

In [21]:
ds='Nonliving-26'
epoch=450
n_test_sample=2600
iid_emb = f'../cache/{ds}/resnet50_1-{epoch}/scratch/id_m1-{epoch}_d1.pkl'
novel_emb = f'../cache/{ds}/resnet50_1-{epoch}/scratch/od_pnovel_m1-{epoch}_cclean-0_n{n_test_sample}.pkl'
corr_emb = f'../cache/{ds}/resnet50_1-{epoch}/scratch/od_psame_m1-{epoch}_cfrost-5_n{n_test_sample}.pkl'

In [22]:
iid_sm = get_conf_scores(iid_emb)
novel_sm = get_conf_scores(novel_emb)
corr_sm = get_conf_scores(corr_emb)

averaged conf: 0.7038018107414246
acc: 0.6948999762535095
label dist: [0.0361, 0.041, 0.0383, 0.0329, 0.0369, 0.0335, 0.0382, 0.0317, 0.051, 0.0409, 0.0375, 0.0463, 0.0371, 0.0366, 0.0431, 0.043, 0.0284, 0.0373, 0.0354, 0.0398, 0.0422, 0.0392, 0.0292, 0.0365, 0.0371, 0.0508]

averaged conf: 0.5417004227638245
acc: 0.3407692313194275
label dist: [0.03230769230769231, 0.038461538461538464, 0.05, 0.023846153846153847, 0.02576923076923077, 0.03769230769230769, 0.04730769230769231, 0.022692307692307692, 0.03576923076923077, 0.03923076923076923, 0.028076923076923076, 0.05423076923076923, 0.035, 0.036153846153846154, 0.03923076923076923, 0.04153846153846154, 0.03769230769230769, 0.04692307692307692, 0.03769230769230769, 0.041923076923076924, 0.04423076923076923, 0.038461538461538464, 0.028846153846153848, 0.03807692307692308, 0.03807692307692308, 0.06076923076923077]

averaged conf: 0.4134865999221802
acc: 0.21230769157409668
label dist: [0.02346153846153846, 0.008846153846153846, 0.079230769

In [23]:
import ot

In [24]:
import altair as alt
import pandas as pd
import numpy as np

# alt.data_transformers.enable('default', max_rows=None)

In [25]:
iid_hist, _ = np.histogram(iid_sm, bins=[i/10 for i in range(11)])
corr_hist, _ = np.histogram(corr_sm, bins=[i/10 for i in range(11)])
novel_hist, _ = np.histogram(novel_sm, bins=[i/10 for i in range(11)])

In [26]:
iid_dens = iid_hist / sum(iid_hist)
corr_dens = corr_hist / sum(corr_hist)
novel_dens = novel_hist / sum(novel_hist)

In [27]:
def transform_data(dens, group):
    return [{'percent': i, 'conf': ind/10+0.05, 'group': group} for ind, i in enumerate(dens)]

In [28]:
dens = transform_data(iid_dens, 'iid') + transform_data(corr_dens, 'corr') + transform_data(novel_dens, 'novel')

In [29]:

alt.Chart(pd.DataFrame(dens)).mark_bar().encode(
    x=alt.X('conf:Q', bin=True),
    y=alt.Y('percent:Q'),
    column='group:N',
    color='group:N'
).properties(
    width=200,
    height=200
)

/usr/workspace/lu35/anaconda3/envs/ood/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)